In [24]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import webbrowser
from threading import Timer

# Load datasets
files = [
    'tab_negative_rnn_pred.csv',
    'tab_negative_ts_pred.csv',
    'tab_neutral_rnn_pred.csv',
    'tab_neutral_ts_pred.csv',
    'tab_positive_rnn_pred.csv',
    'tab_positive_ts_pred.csv'
]

# Combine all datasets
all_data = pd.concat([pd.read_csv(file) for file in files])

# Map tag values to descriptive labels
tag_labels = {
    'rnn_pos': 'Positive Sentiment - Recurrent Neural Network',
    'rnn_neg': 'Negative Sentiment - Recurrent Neural Network',
    'rnn_neu': 'Neutral Sentiment - Recurrent Neural Network',
    'ts_pos': 'Positive Sentiment - SARIMA Time Series',
    'ts_neg': 'Negative Sentiment - SARIMA Time Series',
    'ts_neu': 'Neutral Sentiment - SARIMA Time Series'
}
all_data['tag_label'] = all_data['tag'].map(tag_labels)

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
server = app.server 

app.layout = html.Div([
    html.H1("Tweet Sentiment Prediction"),

    html.Label("Select the Analysis:"),
    dcc.Dropdown(
        id='tag-filter',
        options=[{'label': label, 'value': tag} for tag, label in tag_labels.items()],
        value=list(tag_labels.keys())[0],  # Initial value
        clearable=False
    ),

    # Big Numbers
    html.Div([
        html.Div([
            html.H3("Forecast: Day 1"),
            html.P(id='forecast-day-1', style={'fontSize': 24, 'margin': '10px 0'})
        ], style={
            'display': 'inline-block', 'width': '30%', 'textAlign': 'center',
            'border': '2px solid #4CAF50', 'borderRadius': '10px', 'padding': '10px', 'boxShadow': '0 4px 8px 0 rgba(0, 0, 0, 0.2)', 'margin': '10px'
        }),

        html.Div([
            html.H3("Forecast: Day 3"),
            html.P(id='forecast-day-3', style={'fontSize': 24, 'margin': '10px 0'})
        ], style={
            'display': 'inline-block', 'width': '30%', 'textAlign': 'center',
            'border': '2px solid #FF9800', 'borderRadius': '10px', 'padding': '10px', 'boxShadow': '0 4px 8px 0 rgba(0, 0, 0, 0.2)', 'margin': '10px'
        }),

        html.Div([
            html.H3("Forecast: Day 7"),
            html.P(id='forecast-day-7', style={'fontSize': 24, 'margin': '10px 0'})
        ], style={
            'display': 'inline-block', 'width': '30%', 'textAlign': 'center',
            'border': '2px solid #F44336', 'borderRadius': '10px', 'padding': '10px', 'boxShadow': '0 4px 8px 0 rgba(0, 0, 0, 0.2)', 'margin': '10px'
        }),
    ], style={'textAlign': 'center', 'padding': '20px'}),

    dcc.Graph(id='time-series-plot')
])

@app.callback(
    [
        Output('forecast-day-1', 'children'),
        Output('forecast-day-3', 'children'),
        Output('forecast-day-7', 'children'),
        Output('time-series-plot', 'figure')
    ],
    [Input('tag-filter', 'value')]
)
def update_dashboard(selected_tag):
    filtered_data = all_data[all_data['tag'] == selected_tag]

    # Big Numbers
    future_data = filtered_data[filtered_data['type'] == 'future']
    day_1 = f"{round(future_data['forecast'].iloc[0]):,}" if len(future_data) > 0 else 'N/A'
    day_3 = f"{round(future_data['forecast'].iloc[2]):,}" if len(future_data) > 2 else 'N/A'
    day_7 = f"{round(future_data['forecast'].iloc[6]):,}" if len(future_data) > 6 else 'N/A'

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=filtered_data[(filtered_data['type'] != 'future') & (filtered_data['type'] != 'test')]['date'],
        y=filtered_data[(filtered_data['type'] != 'future') & (filtered_data['type'] != 'test')]['actual'],
        mode='lines+markers',
        name='Train',
        line=dict(color='blue')
    ))

    fig.add_trace(go.Scatter(
        x=filtered_data[(filtered_data['type'] != 'future') & (filtered_data['type'] != 'train')]['date'],
        y=filtered_data[(filtered_data['type'] != 'future') & (filtered_data['type'] != 'train')]['actual'],
        mode='lines+markers',
        name='Test',
        line=dict(color='green')
    ))

    fig.add_trace(go.Scatter(
        x=filtered_data['date'],
        y=filtered_data['forecast'],
        mode='lines+markers',
        name='Test (Predictions)',
        line=dict(color='orange')
    ))

    fig.add_trace(go.Scatter(
        x=filtered_data[filtered_data['type'] == 'future']['date'],
        y=filtered_data[filtered_data['type'] == 'future']['forecast'],
        mode='lines+markers',
        name='Predictions Future',
        line=dict(color='red')
    ))

    fig.update_layout(
        title=f"Tweet Sentiment Prediction for: {tag_labels[selected_tag]}",
        xaxis_title="Date",
        yaxis_title="Value",
        legend_title="Legend",
        template="plotly",
        xaxis=dict(tickangle=45),
        margin=dict(t=50, b=50)
    )

    return day_1, day_3, day_7, fig

def open_browser():
    webbrowser.open_new("http://127.0.0.1:8050")

if __name__ == '__main__':
    Timer(1, open_browser).start()
    app.run_server(debug=True, host='127.0.0.1', port=8050)
